In [15]:
from importlib.metadata import version

import tiktoken
import torch
import re

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
import os
os.environ['TIKTOKEN_CACHE_DIR'] = ""


torch version: 2.3.0
tiktoken version: 0.7.0


In [16]:
with open('the-verdict.txt', 'r', encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of characters in file: ", len(raw_text))
print(raw_text[:99])

Total number of characters in file:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Tokenizing

In [17]:
# convert raw text file into a list of strings (tokens)
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4649


Converting to Token ID

In [18]:
unique_words = sorted(list(set(preprocessed)))
print(len(unique_words))

1159


In [19]:
# create a vocabulary using all unique words
# sorted in alphabetic order
vocab = {token:integer for integer,token in enumerate(unique_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)


In [20]:
#combining encoding and decoding
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [21]:
# adding unkown tokens
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(len(all_tokens))
#print last 5 items in vocabulary
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

1161
('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)


In [22]:
# updating tokenizer to read unknown vocab
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int 
                        else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [23]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [24]:
tokenizer.encode(text)

[1160,
 5,
 362,
 1155,
 642,
 1000,
 10,
 1159,
 57,
 1013,
 981,
 1009,
 738,
 1013,
 1160,
 7]

In [25]:
tokenizer.decode(tokenizer.encode(text))
# hello not in vocab - decoded as unknown

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

Byte Pair Encoding

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))


tiktoken version: 0.7.0


In [27]:
import os
import tempfile
from pathlib import Path
import shutil

# Set the environment variable to prevent caching
os.environ['TIKTOKEN_CACHE_DIR'] = ""

# Identify the cache directory
cache_dir = os.path.join(tempfile.gettempdir(), "data-gym-cache")
print(f"Cache directory: {cache_dir}")

# Check if the cache directory exists and clear it if it does
if os.path.exists(cache_dir):
    print("Cache directory found. Attempting to clear it...")
    shutil.rmtree(cache_dir)
    print("Cache directory cleared.")
else:
    print("Cache directory does not exist.")


Cache directory: /tmp/data-gym-cache
Cache directory found. Attempting to clear it...
Cache directory cleared.


In [28]:


import tiktoken


tokenizer = tiktoken.get_encoding("gpt2")


In [29]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [30]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [31]:
text = ("Akwirw ier")
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


Data Sampling - Sliding Window

In [32]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text))


5145


In [33]:
# removing 50 tokens from dataset to create a sample dataset
encoding_sample = encoded_text[50:]

In [34]:
# next word prediction
# x contains inputs
# y contains targets shifter by one position

context_size = 4
x = encoding_sample[:context_size]
y = encoding_sample[1:context_size+1]

print(x,y)

[290, 4920, 2241, 287] [4920, 2241, 287, 257]


In [35]:
# next word prediction:
for i in range(1, context_size+1):
    context = encoding_sample[:i]
    desired = encoding_sample[i]
    print(context, "--->", desired)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


Data Loader

In [36]:
# extracts chunks from the input dataset, and creates another dataset and dataloader
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = [] #input data ID
        self.target_ids = [] # target ID (to be predicted)

        # tokenize
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # sliding window to extract chunks of max length into overlapping sequences
        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i: i+max_length]
            target_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]
        